# Data

In [1]:
import os 

class Dir_List:
        
    def __init__(self,path_,exten):
        self.path_exten = [path_+"\\"+f for f in os.listdir(path_) if f.endswith(".{}".format(exten))]
        self.name_exten = [f for f in os.listdir(path_) if f.endswith(".{}".format(exten))]
        #return self.path_exten,self.name_exten

    def data_list(self,path_,exten):
        self.path_exten = [path_+"\\"+f for f in os.listdir(path_) if f.endswith(".{}".format(exten))]
        self.name_exten = [f for f in os.listdir(path_) if f.endswith(".{}".format(exten))]
        return self.path_exten,self.name_exten


In [2]:
import os
import pandas as pd
from dateutil.parser import parse
from datetime import datetime
# kwargs 는 유지시키고,
# args 에는 하나를 추가한다.


class Data_Load(Dir_List):
    

    
    def __init__(self,remove_col,*args):
        super(Data_Load,self).__init__(*args)
        self.df = pd.DataFrame()
        for pa_,na_ in zip(self.path_exten,self.name_exten):
            df_ = pd.read_excel(pa_).drop(columns="Unnamed: 0")
            df_["Product Name"] = na_
            self.df = pd.concat([self.df,df_]).reset_index(drop=True)
        if remove_col !=None:
            self.df = self.df[self.df[remove_col] != "None"].reset_index(drop=True)
        else:
            pass

    
    def parsing_date(self,target_):
        parsing = [parse(d) for d in self.df[target_]]
        self.df["parse_date"] = parsing
        
        self.df["year"] = [d.strftime("%Y") for d in self.df["parse_date"]]
        self.df["month"] = [d.strftime("%Y-%m") for d in self.df["parse_date"]]
        self.df["day"] = [d.strftime("%Y-%m-%d") for d in self.df["parse_date"]]
        return self.df


In [3]:
import pandas as pd
import kss
from tqdm.notebook import tqdm
from konlpy.tag import Kkma,Okt
okt = Okt()
kkma = Kkma()

class Pos_Process:
    

    
    def __init__(self,df,target_):
        self.df = df
        sent_li = []
        tq = tqdm(range(len(self.df)),desc="Split Sentences")
        for i in tq:
            try:
                sen_ = kss.split_sentences(df[target_][i])
                sent_li.append(sen_)
            except:
                sent_li.append([])
        self.df["sentences"] = sent_li

    def pos_tagging(self):
        tag_li = []
        wo_li = []
        tq = tqdm(range(len(self.df)),desc="Pos Tagging")
        for i in tq:
            sen = self.df["sentences"][i]
            tags_ = []
            wos_ = []
            for s in sen:
                tag_ = okt.pos(s,stem=True)
                word_ = [w[0] for w in tag_]
                tags_.append(tag_)
                wos_.append(word_)
            tag_li.append(tags_)
            wo_li.append(wos_)
        self.df["sen_tag"] = tag_li
        self.df["sen_word"] = wo_li
        return self.df


### Data Loading

In [4]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\coupang"
test_ = Data_Load("contents",path_,"xlsx").parsing_date("date")
test_pos = Pos_Process(test_,"contents").pos_tagging()
test_pos.head()

Split Sentences:   0%|          | 0/43473 [00:00<?, ?it/s]

Pos Tagging:   0%|          | 0/43473 [00:00<?, ?it/s]

,date,id,item,score,contents,liked,Product Name,parse_date,year,month,day,sentences,sen_tag,sen_word
0,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,가격저렴하고 간단하고 손쉽게 사용할수있어 아주 좋습니다 남자들 사용하기 아주 굿입니다,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-11,2020,2020-12,2020-12-11,"[가격저렴하고 간단하고 손쉽게 사용할수있어 아주 좋습니다, 남자들 사용하기 아주 굿...","[[(가격, Noun), (저렴하다, Adjective), (간단하다, Adject...","[[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다], [남자, 들..."
1,2020.12.04,이*정,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,가격대비 예쁘고 좋아요!,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-04,2020,2020-12,2020-12-04,[가격대비 예쁘고 좋아요!],"[[(가격, Noun), (대비, Noun), (예쁘다, Adjective), (좋...","[[가격, 대비, 예쁘다, 좋다, !]]"
2,2020.12.04,황*철,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",4,가격대비 만족합니다.,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-04,2020,2020-12,2020-12-04,[가격대비 만족합니다.],"[[(가격, Noun), (대비, Noun), (만족하다, Adjective), (...","[[가격, 대비, 만족하다, .]]"
3,2020.12.01,박*란,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,저는 아직도 잘쓰고 있어요 산지 좀 되어ㅛ는데도 좋네요\n가볍고,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-01,2020,2020-12,2020-12-01,"[저는 아직도 잘쓰고 있어요, 산지 좀 되어ㅛ는데도 좋네요, 가볍고]","[[(저, Noun), (는, Josa), (아직도, Adverb), (잘쓰다, V...","[[저, 는, 아직도, 잘쓰다, 있다], [산지, 좀, 되어다, ㅛ, 는데도, 좋다..."
4,2020.11.30,유*환,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,약간 탄내가 조금 나요,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-11-30,2020,2020-11,2020-11-30,[약간 탄내가 조금 나요],"[[(약간, Noun), (타다, Verb), (조금, Noun), (나요, Eom...","[[약간, 타다, 조금, 나요]]"


### Dic Loading

In [113]:
import pandas as pd
from IPython.display import display
import ast

from konlpy.tag import Kkma,Okt
okt,kkma = Okt(),Kkma()



class Dic_Loading:

    
    def __init__(self,path,sheet=None):
        self.df = pd.read_excel(path,sheet_name=sheet).fillna("")
        
        ### change colnames
        display(self.df.loc[0:2,:])
        col_ = list(self.df.columns)
        print(col_)
        input_ = list(input("Inset New columns's name in order:").split(","))
        input_ = "key,attr,attr_sim,attr_combi,count,pos,neg".split(",") if input_==[""] else input_
        col_dic = {}
        for i in range(len(input_)):
            col_dic[col_[i]] = input_[i]
        self.df = self.df.rename(columns=col_dic)
        
        ### to list 
        for col in self.df.columns:
            try:
                self.df[col] = self.df[col].map(lambda x: ast.literal_eval(x))
            except (ValueError,SyntaxError):
                try:
                    self.df[col] = self.df[col].map(lambda x: x.split("#")[1:] if "#" in x else x.split(",") if ","in x else [x])
                except (TypeError,SyntaxError,AttributeError):
                    self.df[col] = self.df[col].map(lambda x : [x])


    def load(self,path,sheet=None):
        self.df = pd.read_excel(path,sheet_name=sheet)
        display(self.df.head())
        col_ = list(self.df.columns)
        print(col_)
        input_ = list(input("Inset New columns's name in order:").split(","))
        col_dic = {}
        for i in range(len(input_)):
            col_dic[col_[i]] = input_[i]
        self.df = self.df.rename(columns=col_dic)
        return self.df
    
    

    
    def pos_tag(self,Version,Tag_dic,Tag_dic_n):
    
        
        def pos_tag_(List_se,Version):
            # list_se = ["바람도 쎄고","바람 세기 충분"]

            if Version=="kkma":
                pos_tag = [kkma.pos(f) for f in List_se]
            elif Version =="okt":
                pos_tag = [ okt.pos(f,stem=True) for f in List_se]
            return pos_tag

        def tag_extract_(List_ta,Tag_dic,Stack_ver=None):
            # list_ta = [[('바람', 'Noun'), ], [('드라이기', 'Noun'), ('바람', 'Noun'), ('세기', 'Noun')]

            dic_tag = {}
            for key_,values_ in Tag_dic.items():
                for value_ in values_:
                    dic_tag[value_] = key_

            out_li = []
            for lt in List_ta:
                tag_ex = [i[0] for i in lt if i[1] in list(dic_tag.keys())]
                if Stack_ver==None:
                    out_li.append(tag_ex)
                elif Stack_ver=="extend":
                    out_li.extend(tag_ex)
            return out_li 

        def delete_duple_(list_):
            tass = []
            for i in list_:
                if i in tass:
                    pass
                else:
                    tass.append(i)
            return tass
        
        def removal_list(Tester_,removal):
            for re in removal:
                try:
                    Tester_.remove(re)
                except:
                    pass
            return Tester_
        
        def removal_funda(fun_,remove_):
            for fu_ in fun_:
                if fu_ in remove_:
                    fun_.remove(fu_)
            return fun_
        
        def remove_list_pos_neg(pos_,remove_):
            remo_2 =[]
            for remo_ in remove_:
                remo_2.extend(remo_)
            pos_2 = []
            for remo_ in remo_2:
                for po_ in pos_:
                    if remo_ in po_:
                        po_.remove(remo_)
                        pos_2.append(po_)
            pos_2 = removal_list(delete_duple_(pos_2),[[]])
            return pos_2
        
        
        
        #### MAIN ###############
        tq = tqdm(range(len(self.df)),desc="Pos Tagging")
        
        self.dic = {}
        for i in tq:
            # key
            sk = tag_extract_(pos_tag_(self.df["key"][i],Version),Tag_dic)
            sk2 = [["".join(ke)] for ke in sk] 
            sk3 = []
            for sk_ in sk:
                for sk__ in sk_:
                    sk3.append([sk__])
            key = sk + sk2 + sk3
            
            key = delete_duple_(key)
            removal2 = [['기능'],['감'], ['/'], ['및'],[]]
            key = removal_list(key,removal2)
            
            # fundamental = attr + attr_sim + attr_combi
            sa = tag_extract_(pos_tag_(self.df["attr"][i],Version),Tag_dic)
            sas =tag_extract_(pos_tag_(self.df["attr_sim"][i],Version),Tag_dic)
            #sac = tag_extract_(pos_tag_(self.df["attr_combi"][i],Version),Tag_dic)
            
            funda = sa + sas# + sac
            funda = delete_duple_(funda)
            
            
            # positive & negative
            positive = tag_extract_(pos_tag_(self.df["pos"][i],Version),Tag_dic)
            negative = tag_extract_(pos_tag_(self.df["neg"][i],Version),Tag_dic)
            
            funda = removal_funda(funda,positive)
            funda = removal_funda(funda,negative)
            
            
            # unioun intersetion 
            spn = set(tag_extract_(pos_tag_(self.df["pos"][i],Version),Tag_dic_n,"extend"))
            snn = set(tag_extract_(pos_tag_(self.df["neg"][i],Version),Tag_dic_n,"extend"))
            
            noun_inter = [[ni] for ni in list(spn.intersection(snn))]
            noun_union = [[ni] for ni in list(spn.union(snn))]
            
            # remove stopwords
            removal = [['것'],['이'], ['왜'],['더'],[]]
            removal_pn = removal + key + funda
            
            
            key = removal_list(key,removal)
            funda = removal_list(funda,removal)
            positive = remove_list_pos_neg(positive,removal_pn)
            negative = remove_list_pos_neg(negative,removal_pn)
            
            
            
            noun_inter = removal_list(noun_inter,removal)
            noun_union = removal_list(noun_union,removal)

            

            
            
            dic_loop = {"key":key,"fundamental":funda,"positive":positive,"negative":negative,
                        "noun_inter":noun_inter,"noun_union":noun_union}
            self.dic[self.df["attr"][i][0]] = dic_loop
        return self.dic

In [90]:
path = r"C:\Users\Samsung\Desktop\Project\이희정 교수님\project\dic_data\드라이기_20201207.xlsx"
dicdf = Dic_Loading(path,"data_").df
dicdf.head()

,Unnamed: 0,Product Attribute 측면 속성,유사 키워드(attr_sim),토큰화 방식으로 각 속성을 나타내는 키워드 조합,count,긍정,부정
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이 ...","단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐인..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아여...,괴물 토네이도 이런 표현은좀 오바네요


['Unnamed: 0', 'Product Attribute 측면 속성', '유사 키워드(attr_sim)', '토큰화 방식으로 각 속성을 나타내는 키워드 조합', 'count', '긍정', '부정']
Inset New columns's name in order:


,key,attr,attr_sim,attr_combi,count,pos,neg
0,[바람],[바람 세기],"[바람 강도 , 바람 파워 , 드라이기 파워 , 건조 속도]","[바람 세기, 바람 강하다, 바람 세다, 바람 쎄다, 바람 약하다, 바람 파워, 건...",[20916],"[바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도...","[바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바..."
1,[바람],[풍량 조절 ],"[풍속 조절 , 바람강도 조절 , 강도 조절 , 바람 세기 조절]","[풍량 조절, 풍속 조절, 바람 강도 조절, 강도 조절, 바람 세기 조절]",[1002],"[바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조...","[단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약..."
2,[바람],[바람 회전 ],"[토네이도 , 회전 바람]","[바람 회전, 토네이도]",[28],[바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아...,[괴물 토네이도 이런 표현은좀 오바네요]
3,[온도],[냉풍 ],"[차가운 바람 , 찬바람]","[냉풍, 차가운 바람, 찬바람, 바람 차갑다]",[2476],"[가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요...","[냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이..."
4,[온도],[온풍 ],"[따뜻한 바람 , 부드러운 바람]","[온풍, 따뜻하다 바람, 부드럽다 바람]",[2529],"[파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 ...","[온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로..."


In [114]:
td1 = {
    "Noun":["NNG","NNP","NNM","Noun"],
    "Verb":['VV', 'VA', 'VXV', 'VCN',"Verb"],
    "Adverb":["MAG","MAC","Adverb"],
    "Adjective":["XPV","XSM","XSO","XR","Adjective"],
    "Unknown":['UN', 'UV', 'UE',"Unknown"],
    "ETC":["EMO","OL","ETC"]
}
td2 = {
    "Noun":["NNG","NNP","NNM","Noun"]
}

In [115]:
dic = Dic_Loading(path,"data_").pos_tag("okt",td1,td2)

,Unnamed: 0,Product Attribute 측면 속성,유사 키워드(attr_sim),토큰화 방식으로 각 속성을 나타내는 키워드 조합,count,긍정,부정
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이 ...","단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐인..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아여...,괴물 토네이도 이런 표현은좀 오바네요


['Unnamed: 0', 'Product Attribute 측면 속성', '유사 키워드(attr_sim)', '토큰화 방식으로 각 속성을 나타내는 키워드 조합', 'count', '긍정', '부정']
Inset New columns's name in order:


Pos Tagging:   0%|          | 0/70 [00:00<?, ?it/s]

In [138]:
dic["가성비"]

{'key': [['가격', '및', '혜택'], ['가격및혜택'], ['가격'], ['혜택']],
 'fundamental': [['가격', '대비', '가치']],
 'positive': [['성능', '생각', '하다', '좋다'],
  ['좋다'],
  ['강풍', '만족하다'],
  ['바람', '자다', '나오다', '좋다'],
  ['쓸다', '하다'],
  ['완전', '만족'],
  ['상품', '질', '너무', '좋다', '거', '같다']],
 'negative': [['별로', '이다']],
 'noun_inter': [['대비'], ['가격']],
 'noun_union': [['짱짱'],
  ['짱'],
  ['완전'],
  ['만족'],
  ['비'],
  ['최곱'],
  ['탑'],
  ['최고'],
  ['대비'],
  ['갑'],
  ['별로'],
  ['듯해'],
  ['귿'],
  ['생각'],
  ['쵝오'],
  ['질'],
  ['상품'],
  ['바람'],
  ['가격'],
  ['가성'],
  ['강풍'],
  ['거'],
  ['성능']]}

# Evaluation 

In [27]:
def match_result_(Dic_lists,Ta_lists):
    
    def match_f_(dic_list,Ta_list):
        if all(elem in Ta_list for elem in dic_list):    
            return 1,dic_list
        else:
            return 0,[""]
    
    score = list(map(lambda x:match_f_(x,Ta_lists)[0] ,Dic_lists))
    matching = list(map(lambda x:match_f_(x,Ta_lists)[1] ,Dic_lists))
    match_li = []
    [match_li.extend(em) for em in matching]
    return sum(score)," ".join(match_li)

In [31]:
def model1_match_(Dic,Sen):
    out_li = []
    for key,value in Dic.items():
        list_ = ["fundamental","noun_inter","key","positive","negative","noun_union"]#list(value.keys())
        score_ = [match_result_(value[k],Sen)[0] for k in list_]
        string_ = [match_result_(value[k],Sen)[1] for k in list_]
        i = 0
        if score_[i] != 0:
            out_li.append((key,i,score_))
        else:
            i += 1
            while score_[i] != 0:
                out_li.append((key,i,score_))
                i += 1
                if i ==len(list_):
                    break
    return out_li

In [41]:
test_pos.head()

,date,id,item,score,contents,liked,Product Name,parse_date,year,month,day,sentences,sen_tag,sen_word
0,2020.12.11,황*영,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,가격저렴하고 간단하고 손쉽게 사용할수있어 아주 좋습니다 남자들 사용하기 아주 굿입니다,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-11,2020,2020-12,2020-12-11,"[가격저렴하고 간단하고 손쉽게 사용할수있어 아주 좋습니다, 남자들 사용하기 아주 굿...","[[(가격, Noun), (저렴하다, Adjective), (간단하다, Adject...","[[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다], [남자, 들..."
1,2020.12.04,이*정,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,가격대비 예쁘고 좋아요!,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-04,2020,2020-12,2020-12-04,[가격대비 예쁘고 좋아요!],"[[(가격, Noun), (대비, Noun), (예쁘다, Adjective), (좋...","[[가격, 대비, 예쁘다, 좋다, !]]"
2,2020.12.04,황*철,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",4,가격대비 만족합니다.,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-04,2020,2020-12,2020-12-04,[가격대비 만족합니다.],"[[(가격, Noun), (대비, Noun), (만족하다, Adjective), (...","[[가격, 대비, 만족하다, .]]"
3,2020.12.01,박*란,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,저는 아직도 잘쓰고 있어요 산지 좀 되어ㅛ는데도 좋네요\n가볍고,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-12-01,2020,2020-12,2020-12-01,"[저는 아직도 잘쓰고 있어요, 산지 좀 되어ㅛ는데도 좋네요, 가볍고]","[[(저, Noun), (는, Josa), (아직도, Adverb), (잘쓰다, V...","[[저, 는, 아직도, 잘쓰다, 있다], [산지, 좀, 되어다, ㅛ, 는데도, 좋다..."
4,2020.11.30,유*환,"CKI 가정용 접이식 헤어 드라이어 1200W, CKI-D200, 혼합 색상",5,약간 탄내가 조금 나요,0,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",2020-11-30,2020,2020-11,2020-11-30,[약간 탄내가 조금 나요],"[[(약간, Noun), (타다, Verb), (조금, Noun), (나요, Eom...","[[약간, 타다, 조금, 나요]]"


In [133]:
import pandas as pd
from tqdm.notebook import tqdm

class Model1:
    
    def __init__(self,df):
        date_li,id_li,product_li,sen_li,score_li,liked_li = [],[],[],[],[],[]
        tq_ = tqdm(range(len(df)),desc="Preprocessing")
        for i in tq_:
            for sen in df.loc[i,"sen_word"]:
                date_li.append(df.loc[i,"date"])
                id_li.append(df.loc[i,"id"])
                product_li.append(df.loc[i,"Product Name"])
                score_li.append(df.loc[i,"score"])
                liked_li.append(df.loc[i,"liked"])
                sen_li.append(sen)
        da = [date_li,id_li,product_li,score_li,liked_li,sen_li]
        self.df = pd.DataFrame(da).T.rename(columns={0:"date",1:"id",2:"Product Name",3:"score",4:"liked",5:"sentence"})
    
    def Model1_(self,dic,range_=None):

        def match_result_(Dic_lists,Ta_lists):
            def match_f_(dic_list,Ta_list):
                if all(elem in Ta_list for elem in dic_list):    
                    return 1,dic_list
                else:
                    return 0,[""]

            score = list(map(lambda x:match_f_(x,Ta_lists)[0] ,Dic_lists))
            matching = list(map(lambda x:match_f_(x,Ta_lists)[1] ,Dic_lists))
            match_li = []
            [match_li.extend(em) for em in matching]
            return sum(score)," ".join(match_li)

        def model1_match_(Dic,Sen):
            out_li = []
            for key,value in Dic.items():
                list_ = ["fundamental","noun_inter","key","positive","negative","noun_union"]#list(value.keys())
                score_ = [match_result_(value[k],Sen)[0] for k in list_]
                string_ = [match_result_(value[k],Sen)[1] for k in list_]
                i = 0
                if score_[i] != 0:
                    out_li.append((key,i,score_))
                else:
                    i += 1
                    while score_[i] != 0:
                        out_li.append((key,i,score_))
                        i += 1
                        if i ==len(list_):
                            break
            return out_li

        ### Main #### 
        model1_li = []
        if range_ != None:
            df2 = self.df.loc[range_[0]:range_[1]]
            tq = tqdm(df2["sentence"],desc="Model Mathcing")
            for sen in tq:
                model1_li.append([(i[0],i[1],i[2]) for i in model1_match_(dic,sen)])
            df2["model1_match"] = model1_li
            return df2
        else:
            tq = tqdm(self.df["sentence"],desc="Model Mathcing")
            for sen in tq:
                model1_li.append([(i[0],i[1],i[2]) for i in model1_match_(dic,sen)])
            self.df["model1_match"] = model1_li
            return self.df

In [134]:
df_ = Model1(test_pos)
df_.df.head()

Preprocessing:   0%|          | 0/43473 [00:00<?, ?it/s]

,date,id,Product Name,score,liked,sentence
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다]"
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[남자, 들, 사용, 하다, 아주, 굿, 이다]"
2,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[가격, 대비, 예쁘다, 좋다, !]"
3,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,0,"[가격, 대비, 만족하다, .]"
4,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[저, 는, 아직도, 잘쓰다, 있다]"


In [135]:
ma = df_.Model1_(dic,[0,1000])
ma.head()

Model Mathcing:   0%|          | 0/1001 [00:00<?, ?it/s]

C:\Users\Samsung\anaconda3\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,date,id,Product Name,score,liked,sentence,model1_match
0,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[가격, 저렴하다, 간단하다, 손쉽다, 사용, 하다, 아주, 좋다]","[(가성비, 1, [0, 1, 1, 1, 0, 1]), (가성비, 2, [0, 1,..."
1,2020.12.11,황*영,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[남자, 들, 사용, 하다, 아주, 굿, 이다]",[]
2,2020.12.04,이*정,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[가격, 대비, 예쁘다, 좋다, !]","[(가성비, 1, [0, 2, 1, 1, 0, 2]), (가성비, 2, [0, 2,..."
3,2020.12.04,황*철,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",4,0,"[가격, 대비, 만족하다, .]","[(가성비, 1, [0, 2, 1, 0, 0, 2]), (가성비, 2, [0, 2,..."
4,2020.12.01,박*란,"coupang_CKI_가정용_접이식_헤어_드라이어_1200W,_CKI-D200,_혼...",5,0,"[저, 는, 아직도, 잘쓰다, 있다]",[]


In [136]:
ma.to_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\test\2\model1_2.xlsx")

In [ ]:
a = ["a","B","c"]
b = [["a"],["b"],["c"]]